In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!pip install underthesea
!pip install gensim

     |████████████████████████████████| 7.5MB 22.8MB/s 
     |████████████████████████████████| 747kB 57.1MB/s 
     |████████████████████████████████| 245kB 55.5MB/s 
     |████████████████████████████████| 51kB 9.6MB/s 
     |████████████████████████████████| 6.7MB 51.5MB/s 
     |████████████████████████████████| 1.3MB 54.5MB/s 
     |████████████████████████████████| 753.2MB 25kB/s 
     |████████████████████████████████| 1.1MB 48.7MB/s 
     |████████████████████████████████| 2.9MB 40.8MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=01e8c0565eefdd9032050d831dc0d45c447f54a6a16d535c075ad1ddf454e547
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f4312d3f4a9dc6ead7c3f3367de4fa90202e6b9784da3b0b7b0eff7777fe3147
  Stored in directory

In [4]:
import os
import re
import gzip
import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import string
from underthesea import word_tokenize
from gensim.utils import simple_preprocess
from gensim.models.wrappers import FastText
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential 
from keras.layers import Dense, Embedding 
from keras.layers import LSTM 
from keras.layers.wrappers import Bidirectional
from keras.layers import Dense, Lambda, dot, Activation, concatenate, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization, AveragePooling1D
import gensim as gs
import tensorflow as tf

import pandas as pd
from gensim.models import FastText

# 1 CHUẨN BỊ DỮ LIỆU

In [7]:
# def create_stopwords(path,original_stopwords):
#   with open(path, encoding="utf-8") as words:
#     return [w[:len(w) - 1] for w in words] + original_stopwords

# original_stopwords = ["tiki","lazada", "shopee"]
# stopwords = create_stopwords('/content/drive/MyDrive/nlp/input/Vietnamese_stopwords/vietnamese-stopwords-dash.txt',original_stopwords)
# np.save("/content/drive/MyDrive/nlp/input/Vietnamese_stopwords/vietnamese-stopwords.npy",stopwords)

vietnamese_stopwords=np.load('/content/drive/MyDrive/nlp/input/Vietnamese_stopwords/vietnamese-stopwords.npy',allow_pickle='TRUE').tolist()

In [8]:
names = ["classes", "content"]
data1sao=pd.read_csv('/content/drive/MyDrive/nlp/input/datacsv2/1sao.csv', names=names)
data2sao=pd.read_csv('/content/drive/MyDrive/nlp/input/datacsv2/2sao.csv', names=names)
data3sao=pd.read_csv('/content/drive/MyDrive/nlp/input/datacsv2/3sao.csv', names=names)
data4sao=pd.read_csv('/content/drive/MyDrive/nlp/input/datacsv2/4sao.csv', names=names)
dataset=pd.concat([data1sao,data2sao,data3sao,data4sao])


In [9]:
data_label_1 = dataset[dataset['classes']==1]['content'].values
data_label_2 = dataset[dataset['classes']==2]['content'].values
data_label_3 = dataset[dataset['classes']==3]['content'].values
data_label_4 = dataset[dataset['classes']==4]['content'].values


In [10]:
# data_label_1 = data_label_1[:5]
data_label_1[:5]

array(['Vỏ ốp tai nghe không có đèn báo hiệu. Chất lượng âm thanh kém. Tín hiệu chập chờn.',
       'Bị treo khi khởi động lại bằng App Tether. Phải khởi động thủ công bằng tay (tắt và mở lại nguồn) , Khởi động hơn 02 phút , khá là lâu so với các thiết bị router tầm trung khác. (kiểm tra cùng các hãng technicolor 7200 , tplink WR840N) Mặc dù quảng cáo là chuẩn N max 450Mb nhưng chất lượng chậm còn thua các router chuẩn N max 300Mb. (kiểm tra cùng đường truyền 20Mb , chắc không đạt chuẩn rồi) Băng tần 2.4G thì phát quá yếu , cách 4m và qua 1 lớp cửa gỗ thôi đã mất 3/4 cột sóng (đo bằng điện thoại samsung) Nên : trong cùng tầm giá 500k nên chọn router AC băng tần kép có khi tận dụng được băng tần 5G lại ổn hơn. Hoặc chọn router chuẩn N max 300M là hợp lý.',
       'Chuột dùng rất ổn trong gần 4 tháng đầu nhưng sau đó đột ngột không sử dụng được nữa (đêm hôm trước dùng bt , sáng hôm sau bật lên đèn nhưng chuột không di chuyển) Mình mua 2 con , 1 ở Tiki và 1 ở Shopee , cách nhau 1 tuần , v

In [11]:
data_train = dataset['content'].values
type(data_train)
# data_train[:2]

numpy.ndarray

In [15]:
def tao_tu_dien(data,stopwords):
  data = pd.read_csv('/content/drive/MyDrive/nlp/input/tudien.txt',encoding='utf8')
  internet_dict=pd.DataFrame(data)
  my_dict = {v: 1 for v in internet_dict['tuvung']}
  for sentence in data:
    text = sentence.translate(str.maketrans('', '', string.punctuation))
    text = text.translate(str.maketrans('', '', string.digits))
    text = text.translate(string.punctuation)
    text = text.lower().split()
    text = " ".join(text)
    sentence = " ".join(simple_preprocess(text))
    sentence = [word for word in word_tokenize(sentence.lower(), format="text").split() if word not in stopwords]
    for word in sentence:
      if(word not in my_dict):
        my_dict[word] = 1
  return my_dict

mydict = tao_tu_dien(data_train,vietnamese_stopwords)


In [ ]:
mydict

In [ ]:
# mydict = np.load('/content/drive/MyDrive/Colab Notebooks/Report2NLP/INPUT_NLP3/dict/word_2_dict_level1_NLP3.npy',allow_pickle='TRUE').item()
# type(mydict)

In [27]:
len_my_dict=len(mydict)

In [28]:
my_dict_array = [sentence for sentence in mydict]

In [ ]:
my_dict_array

In [30]:
def tao_nhieu(sentence):
  so_nhieu = np.random.randint(3,6)
  for i in range(so_nhieu):
    left = np.random.randint(0,2)
    index_words = np.random.randint(0,len_my_dict)
    if left ==1 :
      sentence = str(my_dict_array[index_words]) + ' ' + sentence
    else:
      sentence = sentence + ' ' + str(my_dict_array[index_words] )
  return sentence

In [31]:
def reverse_sentences(sentence):
  sentence = sentence.split(' ')
  sentence.reverse()
  return ' '.join(x for x in sentence)

In [32]:
def lanvung(sentence):
  sentence_arr = sentence.split(' ')
  len_sen = len(sentence_arr)
  if len_sen < 4:
    return sentence
  time_lan = np.random.randint(1,5)
  dict_time = {}
  for i in range(time_lan):
    while True:
      current_index = np.random.randint(0,len_sen)
      if current_index not in dict_time:
        dict_time[current_index] =1 
        sentence_arr[current_index] = (sentence_arr[current_index] + ' ' ) * np.random.randint(2,6)
        break
      else :
        continue
  return ' '.join(x for x in sentence_arr)

In [33]:
data_label_1

array(['Vỏ ốp tai nghe không có đèn báo hiệu. Chất lượng âm thanh kém. Tín hiệu chập chờn.',
       'Bị treo khi khởi động lại bằng App Tether. Phải khởi động thủ công bằng tay (tắt và mở lại nguồn) , Khởi động hơn 02 phút , khá là lâu so với các thiết bị router tầm trung khác. (kiểm tra cùng các hãng technicolor 7200 , tplink WR840N) Mặc dù quảng cáo là chuẩn N max 450Mb nhưng chất lượng chậm còn thua các router chuẩn N max 300Mb. (kiểm tra cùng đường truyền 20Mb , chắc không đạt chuẩn rồi) Băng tần 2.4G thì phát quá yếu , cách 4m và qua 1 lớp cửa gỗ thôi đã mất 3/4 cột sóng (đo bằng điện thoại samsung) Nên : trong cùng tầm giá 500k nên chọn router AC băng tần kép có khi tận dụng được băng tần 5G lại ổn hơn. Hoặc chọn router chuẩn N max 300M là hợp lý.',
       'Chuột dùng rất ổn trong gần 4 tháng đầu nhưng sau đó đột ngột không sử dụng được nữa (đêm hôm trước dùng bt , sáng hôm sau bật lên đèn nhưng chuột không di chuyển) Mình mua 2 con , 1 ở Tiki và 1 ở Shopee , cách nhau 1 tuần , v

In [34]:
data_nhieu =[tao_nhieu(x) for x in data_label_1]
data_label_1 = np.concatenate((data_nhieu,data_label_1))
data_rever =[ reverse_sentences(x) for x in data_label_1]
data_label_1 = np.concatenate((data_rever,data_label_1))
data_loan =[ lanvung(x) for x in data_label_1]
data_label_1 = np.concatenate((data_label_1,data_loan))

In [35]:
data_nhieu =[ tao_nhieu(x) for x in data_label_2]
data_label_2 = np.concatenate((data_nhieu,data_label_2))
data_rever =[ reverse_sentences(x) for x in data_label_2]
data_label_2 = np.concatenate((data_rever,data_label_2))
data_loan =[ lanvung(x) for x in data_label_2]
data_label_2 = np.concatenate((data_label_2,data_loan))

In [36]:
data_nhieu =[ tao_nhieu(x) for x in data_label_3]
data_label_3 = np.concatenate((data_nhieu,data_label_3))
data_rever =[ reverse_sentences(x) for x in data_label_3]
data_label_3 = np.concatenate((data_rever,data_label_3))
data_loan =[ lanvung(x) for x in data_label_3]
data_label_3 = np.concatenate((data_label_3,data_loan))

In [37]:
data_nhieu =[ tao_nhieu(x) for x in data_label_4]
data_label_4 = np.concatenate((data_nhieu,data_label_4))
data_rever =[ reverse_sentences(x) for x in data_label_4]
data_label_4 = np.concatenate((data_rever,data_label_4))
data_loan =[ lanvung(x) for x in data_label_4]
data_label_4 = np.concatenate((data_label_4,data_loan))

In [38]:
label_1 = np.ones(len(data_label_1))*1
label_2 = np.ones(len(data_label_2))*2
label_3 = np.ones(len(data_label_3))*3
label_4 = np.ones(len(data_label_4))*4

In [39]:
data_train = np.concatenate((data_label_1,data_label_2,data_label_3,data_label_4))
labels =  np.concatenate((label_1,label_2,label_3,label_4))

In [40]:
(data_train.shape,labels.shape)

((294752,), (294752,))

In [ ]:
# lưu dict của data khi chưa tạo nhiều
# np.save('/content/drive/MyDrive/Colab Notebooks/Report2NLP/INPUT_NLP3/dict/word_2_dict_level1_NLP3.npy', mydict) 
# mydict = np.load('/content/drive/MyDrive/Colab Notebooks/Report2NLP/INPUT_NLP3/dict/word_2_dict_level1_NLP3.npy',allow_pickle='TRUE').item()
# type(mydict)

**Tạo model embeding fasttext**

In [43]:
# tạo input for embeding
def create_input_for_fasttext(text,stopwords):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.translate(str.maketrans('', '', string.digits))
    text = text.translate(string.punctuation)
    text = text.lower().split()
    text = " ".join(text)
    sentence = " ".join(simple_preprocess(text))
    sentence = [word for word in word_tokenize(sentence.lower(), format="text").split() if word not in stopwords]
    return [word for word in sentence if word != ""]


In [44]:
# input_sentences = [create_input_for_fasttext(str(sentence),vietnamese_stopwords) for sentence in data_train]

In [26]:
len(data_train)

36844

In [47]:
len(input_sentences)

294752

In [48]:
type(input_sentences)
# lưu cái input_sentences lại vì nó chạy quá lâu.
np.save("/content/drive/MyDrive/nlp/input/input_sentences.npy",input_sentences)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [49]:
input_sentences= np.load("/content/drive/MyDrive/nlp/input/input_sentences.npy",allow_pickle='TRUE').tolist()

In [50]:
len(input_sentences)

294752

In [ ]:
input_sentences

In [ ]:
# input_sentences
# data_ft = np.concatenate((data_label_1[0:10000],data_label_2[0:10000],data_label_3[0:10000],data_label_4[0:10000]))
# data_ft = [x.split() for x in data_ft ]

In [52]:
model_fasttext = FastText(size=350, window=15, min_count=50, workers=4, sg=1)
model_fasttext.build_vocab(input_sentences)
model_fasttext.train(input_sentences, total_examples = model_fasttext.corpus_count, epochs = model_fasttext.iter)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
type(model_fasttext)

gensim.models.fasttext.FastText

In [ ]:
# model_fasttext.wv.save("/content/drive/MyDrive/Colab Notebooks/Report2NLP/INPUT_NLP3/model_embeding/model_fasttext_keyedvector_350dim.model")
# model_fasttext1 =gs.models.keyedvectors.KeyedVectors.load("/content/drive/MyDrive/Colab Notebooks/Report2NLP/INPUT_NLP3/model_embeding/model_fasttext_keyedvector_350dim.model")


In [ ]:
type(model_fasttext1)

gensim.models.keyedvectors.FastTextKeyedVectors

In [55]:
# model_fasttext.save("/content/drive/MyDrive/nlp/input/model_fasttext_350dim.model")
model_fasttext=gs.models.FastText.load("/content/drive/MyDrive/nlp/input/model_fasttext_350dim.model")

In [56]:
type(model_fasttext)

gensim.models.fasttext.FastText

# 2 Tạo từ điển, embeding data

**Tạo từ điển level 2 -> pad_sequence,encode label  -> tạo vector embeding cho dữ liệu dựa vào từ điển.level 2**

In [57]:
mydict_level2_nhieu = tao_tu_dien(data_train,vietnamese_stopwords)

In [59]:
len(mydict_level2_nhieu)

29199

In [60]:

# lưu dict của data khi chưa tạo nhiều
# np.save('/content/drive/MyDrive/nlp/input/word_2_dict_level2_NLP3.npy', mydict_level2_nhieu) 
mydict_level2_nhieu = np.load('/content/drive/MyDrive/nlp/input/word_2_dict_level2_NLP3.npy',allow_pickle='TRUE').item()
type(mydict_level2_nhieu)

dict

In [61]:
len(mydict_level2_nhieu)


# """ đi tìm cái len của cái câu nào mà xuất hiện nhiều nhất 
# thì lấy len của câu đó làm giá trị maxword trong câu nếu nó lớn
# quá thì có thể bẻ đôi."""


29199

In [80]:
# fix độ dài cho câu
max_words=350
X = [[mydict_level2_nhieu[word] for word in sentence if(word in mydict_level2_nhieu)] for sentence in data_train]
X_train = sequence.pad_sequences(X,max_words)
Y = labels
le = LabelEncoder()
Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)
Y_train = keras.utils.to_categorical(Y, num_classes=4)

In [81]:
(X_train.shape,Y_train.shape)

((294752, 350), (294752, 4))

In [82]:
def minimum(a, b): 
    if a <= b: 
        return a 
    else: 
        return b 

In [83]:
# embedding matrix
print('preparing embedding matrix...')


MAX_NB_WORDS = 100000
words_not_found = []
embedding_dim = 350 # The dimension of word embeddings
nb_words = minimum(MAX_NB_WORDS, len(mydict_level2_nhieu))


# Với mỗi từ trong câu, lưu lại word vector phụ vụ để huấn luyện mô hình
embedding_matrix = np.zeros((nb_words, embedding_dim))
for word, i in mydict_level2_nhieu.items():
  try:
    if i >= nb_words:
      continue
    embedding_vector = model_fasttext2[word]
    if (embedding_vector is not None) and len(embedding_vector) > 0:
      embedding_matrix[i] = embedding_vector

    else:
      words_not_found.append(word)
  except:
    words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 29199


In [84]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train,test_size=0.2)

In [85]:
((x_train.shape,y_train.shape),(y_test.shape,x_test.shape))

(((235801, 350), (235801, 4)), ((58951, 4), (58951, 350)))

# 3 MODEL

In [86]:
import tensorflow as tf
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


max_words=350
patience = 50
base_path='/content/drive/MyDrive/nlp/output'
early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                              patience=int(patience/4), verbose=1)

trained_models_path = base_path + 'BI_LSTM_CNN'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'

model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                   save_best_only=True)

callback = [model_checkpoint, early_stop, reduce_lr]

# callback = tf.keras.callbacks.ModelCheckpoint(
    # filepath="/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/model_predict/model12899.hdf5", 
    # verbose=1, 
    # # save_weights_only=True,
    # save_freq="epoch")

model=Sequential()
model.add(Embedding(len(mydict_level2_nhieu),
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=max_words))

model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Dropout(0.5))

model.add(Conv1D(32,3,activation='relu'))
model.add(MaxPooling1D())

model.add(Conv1D(64,3,activation='relu'))
model.add(MaxPooling1D())

model.add(Conv1D(128,3,activation='relu'))
model.add(MaxPooling1D())

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4,activation='softmax'))

model.compile(loss = 'categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 350, 350)          10219650  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 350, 1024)         3534848   
_________________________________________________________________
dropout_4 (Dropout)          (None, 350, 1024)         0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 348, 32)           98336     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 174, 32)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 172, 64)           6208      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 86, 64)           

In [88]:
train_history = model.fit(
    x_train, y_train,
    batch_size=512,
    epochs=5,
    callbacks=callback,
    validation_data = (x_test, y_test)
)


Epoch 1/5
461/461 [==============================] - 663s 1s/step - loss: 1.3557 - accuracy: 0.3518 - val_loss: 1.3414 - val_accuracy: 0.3530

Epoch 00001: val_loss improved from inf to 1.34137, saving model to /content/drive/MyDrive/nlp/outputBI_LSTM_CNN.01-0.35.hdf5
Epoch 2/5
461/461 [==============================] - 662s 1s/step - loss: 1.3382 - accuracy: 0.3526 - val_loss: 1.3380 - val_accuracy: 0.3530

Epoch 00002: val_loss improved from 1.34137 to 1.33796, saving model to /content/drive/MyDrive/nlp/outputBI_LSTM_CNN.02-0.35.hdf5
Epoch 3/5
461/461 [==============================] - 653s 1s/step - loss: 1.3371 - accuracy: 0.3526 - val_loss: 1.3378 - val_accuracy: 0.3530

Epoch 00003: val_loss improved from 1.33796 to 1.33784, saving model to /content/drive/MyDrive/nlp/outputBI_LSTM_CNN.03-0.35.hdf5
Epoch 4/5
461/461 [==============================] - 657s 1s/step - loss: 1.3371 - accuracy: 0.3526 - val_loss: 1.3378 - val_accuracy: 0.3530

Epoch 00004: val_loss did not improve from

In [ ]:
plt.plot(train_history.history['accuracy'])
plt.plot(train_history.history['val_accuracy'])
plt.title('Train History')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# 4 Predict